In [12]:
import os
import pandas as pd
import requests
import json

def fetch_data_from_api(api_url, headers=None, params=None):
    try:
        response = requests.get(api_url, headers=headers, params=params)
        response.raise_for_status()
        content_type = response.headers.get('Content-Type')
        if 'application/json' in content_type:
            return response.json(), None
        else:
            return None, response.text
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None, None

def extract_json_after_next_data_tag(file_content):
    try:
        json_start = file_content.find('_NEXT_DATA__" type="application/json">') + len('_NEXT_DATA__" type="application/json">')
        if json_start == -1:
            raise ValueError("Start marker not found in the file content")
        json_content = file_content[json_start:]
        json_end = json_content.find('</script>')
        if (json_end == -1):
            raise ValueError("End marker not found in the file content")
        json_content = json_content[:json_end].strip()
        json_data = json.loads(json_content)
        return json_data, json_content
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        return None, None
    except Exception as e:
        print(f"An error occurred while extracting JSON: {e}")
        return None, None

def save_json_to_file(json_content, output_file):
    try:
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(json_content)
        print(f"JSON content successfully saved to {output_file}")
    except Exception as e:
        print(f"An error occurred while saving JSON to file: {e}")

# Load Excel file and create the JSON_DATA directory
excel_file_path = "C:\\Users\\Sunil\\OneDrive\\Desktop\\Crickbuzz_data\\API_EXTRACTION\\Recent_Matches_with_API_Name.xlsx"
output_folder = "JSON_DATA"
os.makedirs(output_folder, exist_ok=True)

# Read the Excel file
df = pd.read_excel(excel_file_path)

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    api_url = row['final_api']
    file_name = row['api_name']
    json_output_file = os.path.join(output_folder, f"{file_name}.json")
    
    # Check if the file already exists
    if os.path.exists(json_output_file):
        print(f"File {json_output_file} already exists. Skipping...")
        continue
    
    json_data, html_content = fetch_data_from_api(api_url)
    
    if html_content:
        json_data, json_content = extract_json_after_next_data_tag(html_content)
        if json_content:
            save_json_to_file(json_content, json_output_file)
    else:
        if json_data:
            json_content = json.dumps(json_data, indent=4)
            save_json_to_file(json_content, json_output_file)

print("JSON extraction and saving process completed.")


File JSON_DATA\gloucestershire-vs-essex-south-group.json already exists. Skipping...
File JSON_DATA\hampshire-vs-surrey-south-group.json already exists. Skipping...
File JSON_DATA\lancashire-vs-durham-north-group.json already exists. Skipping...
File JSON_DATA\northamptonshire-vs-derbyshire-north-group.json already exists. Skipping...
File JSON_DATA\yorkshire-vs-worcestershire-north-group.json already exists. Skipping...
File JSON_DATA\durham-vs-birmingham-bears-north-group.json already exists. Skipping...
File JSON_DATA\glamorgan-vs-surrey-south-group.json already exists. Skipping...
File JSON_DATA\middlesex-vs-kent-south-group.json already exists. Skipping...
File JSON_DATA\nottinghamshire-vs-northamptonshire-north-group.json already exists. Skipping...
File JSON_DATA\somerset-vs-essex-south-group.json already exists. Skipping...
File JSON_DATA\sussex-vs-gloucestershire-south-group.json already exists. Skipping...
File JSON_DATA\worcestershire-vs-lancashire-north-group.json already e

In [35]:
import os
import pandas as pd
import json

# Path to the folder containing JSON files
json_folder_path = 'JSON_DATA'

# List to hold all DataFrames
all_dataframes = []

# Iterate through all JSON files in the folder
for json_file in os.listdir(json_folder_path):
    if json_file.endswith('.json'):
        # Construct the full file path
        file_path = os.path.join(json_folder_path, json_file)
        
        # Load the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Extract the relevant details
        series_id = data.get('query', {}).get('seriesId')
        match_id = data.get('query', {}).get('matchId')
        innings = data.get('props', {}).get('appPageProps', {}).get('data', {}).get('content', {}).get('innings', [])
        
        # Create an empty list to store all batsman details for this JSON file
        all_batsmen = []
        
        # Iterate over each inning in the innings list
        for inning in innings:
            # Check if 'inningBatsmen' is a key in the inning dictionary
            if 'inningBatsmen' in inning:
                batsmen = inning['inningBatsmen']
                # Add seriesId and matchId to each batsman's dictionary
                for batsman in batsmen:
                    batsman['seriesId'] = series_id
                    batsman['matchId'] = match_id
                # Extend the all_batsmen list with details from the current inning
                all_batsmen.extend(batsmen)
        
        if all_batsmen:
            # Convert the details to a pandas DataFrame
            df = pd.DataFrame(all_batsmen)
            
            # Extract player details into separate columns if the key exists
            if 'player' in df.columns:
                df['playerId'] = df['player'].apply(lambda x: x.get('id') if x else None)
                df['playerLongName'] = df['player'].apply(lambda x: x.get('longName') if x else None)
                df['playerBattingStyles'] = df['player'].apply(lambda x: x.get('battingStyles') if x else None)
                df = df.drop(columns=['player'])
            
            # Extract bowler details into separate columns if the key exists
            if 'dismissalBowler' in df.columns:
                df['bowlerId'] = df['dismissalBowler'].apply(lambda x: x.get('id') if x else None)
                df['bowlerLongName'] = df['dismissalBowler'].apply(lambda x: x.get('longName') if x else None)
                df['bowlingStyles'] = df['dismissalBowler'].apply(lambda x: x.get('bowlingStyles') if x else None)
                df = df.drop(columns=['dismissalBowler'])
            
            # Reorder the columns to ensure seriesId and matchId are first
            # and bowler details are after dismissalType
            columns_order = ['seriesId', 'matchId', 'playerId', 'playerLongName', 'playerBattingStyles'] + \
                            [col for col in df.columns if col not in ['seriesId', 'matchId', 'playerId', 'playerLongName', 'playerBattingStyles', 'bowlerId', 'bowlerLongName', 'bowlingStyles']] + \
                            ['dismissalType', 'bowlerId', 'bowlerLongName', 'bowlingStyles']
            df = df[columns_order]
            
            # Append the DataFrame to the list
            all_dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame if there is data
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Save the combined DataFrame to an Excel file
    final_df.to_excel('all_batsmen_details.xlsx', index=False)

    # Load the Excel file to verify its content
    df_excel = pd.read_excel('all_batsmen_details.xlsx')
    print(df_excel)
else:
    print("No valid batsman data found in the JSON files.")


C:\Users\Sunil\AppData\Local\Temp\ipykernel_22516\3650371768.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_dataframes, ignore_index=True)


      seriesId  matchId  playerId playerLongName playerBattingStyles  \
0      1410370  1410479     46597      Moeen Ali             ['lhb']   
1      1410370  1410479     65368    Alex Davies             ['rhb']   
2      1410370  1410479    104560    Dan Mousley             ['lhb']   
3      1410370  1410479     67482       Sam Hain             ['rhb']   
4      1410370  1410479    105968  Jacob Bethell             ['lhb']   
...        ...      ...       ...            ...                 ...   
2580   1410370  1410375    102629  Matthew Revis             ['rhb']   
2581   1410370  1410375     71499       Dom Bess             ['rhb']   
2582   1410370  1410375    102861   Jafer Chohan             ['rhb']   
2583   1410370  1410375    102375  Dominic Leech             ['rhb']   
2584   1410370  1410375     88645   Dan Moriarty             ['lhb']   

     playerRoleType battedType  runs  balls  minutes  ...  ballOversUnique  \
0                 P        yes  59.0   32.0     37.0  ...

In [36]:
# Function to safely convert string representation of list to actual list and extract the first element
def extract_first_element(value):
    if isinstance(value, str):
        try:
            # Convert string representation of list to actual list
            value_list = ast.literal_eval(value)
            if isinstance(value_list, list) and len(value_list) > 0:
                return value_list[0]
        except (ValueError, SyntaxError):
            # If conversion fails, just return the value
            return value
    return value

# Apply the function to the relevant columns
df_excel['playerBattingStyles'] = df_excel['playerBattingStyles'].apply(extract_first_element)
df_excel['bowlingStyles'] = df_excel['bowlingStyles'].apply(extract_first_element)

# Drop the specified columns
df_excel = df_excel.drop(columns=['dismissalBatsman', 'dismissalFielders', 'dismissalText', 'dismissalComment', 'fowBalls', 'ballOversActual', 'ballOversUnique', 'ballTotalRuns', 'ballBatsmanRuns', 'videos', 'images', 'currentType', 'dismissalType'])

# Optionally, save the DataFrame to a new CSV file
df_excel.to_csv('all_batsmen_details.csv', index=False)

In [14]:
import os
import pandas as pd
import json

# Path to the folder containing JSON files
json_folder_path = 'JSON_DATA'

# List to hold all DataFrames
all_dataframes = []

# Function to extract bowler details into separate columns
def extract_bowler_info(bowlers_list, key):
    if bowlers_list and isinstance(bowlers_list, list):
        return ', '.join([str(bowler.get(key)) for bowler in bowlers_list if bowler.get(key) is not None])
    return None

# Iterate through all JSON files in the folder
for json_file in os.listdir(json_folder_path):
    if json_file.endswith('.json'):
        # Construct the full file path
        file_path = os.path.join(json_folder_path, json_file)
        
        # Load the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Extract the relevant details
        series_id = data.get('query', {}).get('seriesId')
        match_id = data.get('query', {}).get('matchId')
        innings = data.get('props', {}).get('appPageProps', {}).get('data', {}).get('content', {}).get('innings', [])
        
        # Create an empty list to store all inningOvers details for this JSON file
        all_inningOvers = []
        
        # Iterate over each inning in the innings list
        for inning in innings:
            # Check if 'inningOvers' is a key in the inning dictionary
            if 'inningOvers' in inning:
                inningOvers = inning['inningOvers']
                # Add seriesId and matchId to each over's dictionary
                for over in inningOvers:
                    over['seriesId'] = series_id
                    over['matchId'] = match_id
                # Extend the all_inningOvers list with details from the current inning
                all_inningOvers.extend(inningOvers)
        
        if all_inningOvers:
            # Convert the details to a pandas DataFrame
            df = pd.DataFrame(all_inningOvers)
            
            # Add bowlerId and bowlerLongName columns as strings
            if 'bowlers' in df.columns:
                df['bowlerIds'] = df['bowlers'].apply(lambda x: extract_bowler_info(x, 'id'))
                df['bowlerLongNames'] = df['bowlers'].apply(lambda x: extract_bowler_info(x, 'longName'))
                # Drop the original 'bowlers' column
                df = df.drop(columns=['bowlers'])
            
            # Reorder the columns to ensure seriesId, matchId, bowlerIds, and bowlerLongNames are first
            columns_order = ['seriesId', 'matchId', 'bowlerIds', 'bowlerLongNames'] + \
                            [col for col in df.columns if col not in ['seriesId', 'matchId', 'bowlerIds', 'bowlerLongNames']]
            df = df[columns_order]
            
            # Append the DataFrame to the list
            all_dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame if there is data
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Save the combined DataFrame to a CSV file
    final_df.to_csv('all_bowlers.csv', index=False)

    # Load the CSV to verify its content
    df_csv = pd.read_csv('all_bowlers.csv')
    print(df_csv)
else:
    print("No valid bowler data found in the JSON files.")


      seriesId  matchId bowlerIds bowlerLongNames  overNumber  overRuns  \
0      1410370  1410479     51498   Mohammad Amir           1         2   
1      1410370  1410479     71425    Alex Thomson           2         2   
2      1410370  1410479     81537    Zak Chappell           3        10   
3      1410370  1410479     83427       Pat Brown           4         7   
4      1410370  1410479     51498   Mohammad Amir           5        10   
...        ...      ...       ...             ...         ...       ...   
4314   1410370  1410375     61941    Hayden Walsh           8         5   
4315   1410370  1410375     53162       Josh Cobb           9        11   
4316   1410370  1410375     61941    Hayden Walsh          10         9   
4317   1410370  1410375     65499      Tom Taylor          11        14   
4318   1410370  1410375     61941    Hayden Walsh          12         9   

      overWickets  isComplete  totalBalls  totalRuns  totalWickets  \
0               0        True

In [1]:
df_csv

NameError: name 'df_csv' is not defined